In [14]:
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import seaborn as sns
sns.set()
import dipy.reconst.sfm as sfm
import dipy.reconst.csdeconv as csd
import dipy.reconst.dti as dti
import dipy.core.gradients as grad
import dipy.reconst.cross_validation as xval
import dipy.data as dpd
import utils
import model as mm 
from model import Model, BiExponentialIsotropicModel
from parallelization import parallelization
import csv

In [15]:
sphere = dpd.get_sphere()
sphere = sphere.subdivide(1)

In [16]:
# Load Data
data_dict = utils.read_data()
data = data_dict['seen']['signal'][:, range(2)]
bvals = data_dict['seen']['bvals']
bvecs = data_dict['seen']['bvecs']
delta = data_dict['seen']['delta']
Delta = data_dict['seen']['Delta']
te = data_dict['seen']['TE']
g = data_dict['seen']['g']

In [17]:
'''
data_seen = []
data_unseen = []
te_seen = []
te_unseen = []
gtab_seen = []
gtab_unseen = []

mask = utils.create_shells()
for m in range(data.shape[-1]): # for every voxel
    for j in range(0, len(mask)): # for every TE
        for i in range(1, len(mask[j])): # for every shell besides B0s
            mask_unseen = mask[j][i]
            mask_seen = np.invert(mask_unseen)
            data_seen.append(data[mask_seen, m])
            data_unseen.append(data[mask_unseen, m])
            te_seen.append(te[mask_seen])
            te_unseen.append(te[mask_unseen])
            bvals_seen_temp = bvals[mask_seen]
            bvecs_seen_temp = bvecs[mask_seen]
            delta_seen_temp = delta[mask_seen]
            Delta_seen_temp = Delta[mask_seen]
            gtab_seen.append(grad.gradient_table(bvals_seen_temp, bvecs_seen_temp, big_delta=Delta_seen_temp, small_delta=delta_seen_temp))
            bvals_unseen_temp = bvals[mask_unseen]
            bvecs_unseen_temp = bvecs[mask_unseen]
            delta_unseen_temp = delta[mask_unseen]
            Delta_unseen_temp = Delta[mask_unseen]
            gtab_unseen.append(grad.gradient_table(bvals_unseen_temp, bvecs_unseen_temp, big_delta=Delta_unseen_temp, small_delta=delta_unseen_temp))
'''

'\ndata_seen = []\ndata_unseen = []\nte_seen = []\nte_unseen = []\ngtab_seen = []\ngtab_unseen = []\n\nmask = utils.create_shells()\nfor m in range(data.shape[-1]): # for every voxel\n    for j in range(0, len(mask)): # for every TE\n        for i in range(1, len(mask[j])): # for every shell besides B0s\n            mask_unseen = mask[j][i]\n            mask_seen = np.invert(mask_unseen)\n            data_seen.append(data[mask_seen, m])\n            data_unseen.append(data[mask_unseen, m])\n            te_seen.append(te[mask_seen])\n            te_unseen.append(te[mask_unseen])\n            bvals_seen_temp = bvals[mask_seen]\n            bvecs_seen_temp = bvecs[mask_seen]\n            delta_seen_temp = delta[mask_seen]\n            Delta_seen_temp = Delta[mask_seen]\n            gtab_seen.append(grad.gradient_table(bvals_seen_temp, bvecs_seen_temp, big_delta=Delta_seen_temp, small_delta=delta_seen_temp))\n            bvals_unseen_temp = bvals[mask_unseen]\n            bvecs_unseen_temp

In [18]:
data_seen = []
data_unseen = []
te_seen = []
te_unseen = []
gtab_seen = []
gtab_unseen = []

mask = utils.create_shells()
for m in range(data.shape[-1]): # for every voxel
    for j in range(1, len(mask)): # for every TE
        i = 3
        if j == 5 or j == 7:
            pass
        else:
            mask_unseen = mask[j][i]
            mask_seen = np.invert(mask_unseen)
            data_seen.append(data[mask_seen, m])
            data_unseen.append(data[mask_unseen, m])
            te_seen.append(te[mask_seen])
            te_unseen.append(te[mask_unseen])
            bvals_seen_temp = bvals[mask_seen]
            bvecs_seen_temp = bvecs[mask_seen]
            delta_seen_temp = delta[mask_seen]
            Delta_seen_temp = Delta[mask_seen]
            gtab_seen.append(grad.gradient_table(bvals_seen_temp, bvecs_seen_temp, big_delta=Delta_seen_temp, small_delta=delta_seen_temp))
            bvals_unseen_temp = bvals[mask_unseen]
            bvecs_unseen_temp = bvecs[mask_unseen]
            delta_unseen_temp = delta[mask_unseen]
            Delta_unseen_temp = Delta[mask_unseen]
            gtab_unseen.append(grad.gradient_table(bvals_unseen_temp, bvecs_unseen_temp, big_delta=Delta_unseen_temp, small_delta=delta_unseen_temp))

In [19]:
# 3 shells, 11 TEs, 6 Voxel

In [20]:
alphas = [1e-5]#, 1e-1]#1e-9, 1e-7, 1e-5, 1e-3, 1e-1]
l1_ratios = [0.1]#, 0.9]#, 0.3, 0.5, 0.7, 0.9]

In [21]:
n_variations = len(alphas) * len(l1_ratios)
n_voxels = len(data_seen)
n = n_voxels * n_variations

In [22]:
alpha_grid, l1_ratio_grid = np.meshgrid(alphas, l1_ratios)
alpha_grid = np.reshape(alpha_grid, (n_variations, -1)).squeeze()
l1_ratio_grid = np.reshape(l1_ratio_grid, (n_variations, -1)).squeeze()

In [23]:
alpha = np.repeat(alpha_grid, n_voxels).tolist()
l1_ratio = np.repeat(l1_ratio_grid, n_voxels).tolist()

In [24]:
p = parallelization()

In [25]:
models = p.start(Model, n, gtab_seen*n_variations, sphere=[sphere], isotropic=[BiExponentialIsotropicModel], alpha=alpha, l1_ratio=l1_ratio)

Progress[              0%                      ]
Total Time needed: 0.01min



In [26]:
fits = p.start([i.fit for i in models], n, data_seen*n_variations, te_seen*n_variations)

Parallelization starts on 64 CPUs.
Progress[              0%                      ]

Process Process-195:
Process Process-194:
Process Process-80:
Process Process-75:
Process Process-76:
Process Process-78:
Process Process-69:
Process Process-71:
Process Process-67:
Process Process-79:
Process Process-77:
Process Process-70:
Process Process-72:
Process Process-73:
Process Process-68:
Process Process-74:
Process Process-66:
Process Process-65:
Process Process-193:
Process Process-200:
Process Process-205:
Process Process-204:
Process Process-196:
Process Process-208:
Process Process-203:
Process Process-202:
Process Process-198:
Process Process-199:
Process Process-206:
Process Process-201:
Process Process-197:
Process Process-207:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call l

KeyboardInterrupt: 

  File "parallelization.py", line 74, in worker
  File "parallelization.py", line 74, in worker
  File "parallelization.py", line 74, in worker
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **params_dict)
    result = function[vox](*params, **pa

In [ ]:
predicts = p.start([i.predict for i in fits], n, gtab_unseen*n_variations, te_unseen*n_variations)

In [ ]:
predicts_seen = p.start([i.predict for i in fits], n, gtab_seen*n_variations, te_seen*n_variations)

In [ ]:
filename = 'predictions_seen.csv'
with open(filename, 'w') as f:
   writer = csv.writer(f, delimiter=',')
   writer.writerows(predicts_seen)

In [ ]:
filename = 'predictions.csv'
with open(filename, 'w') as f:
   writer = csv.writer(f, delimiter=',')
   writer.writerows(predicts)

In [ ]:
betas = []
te_params = []
iso_params = []
S0s = []
for i in fits:
    betas.append(i.beta)
    te_params.append(i.te_params)
    iso_params.append(i.iso.params)
    S0s.append([i.S0])

In [ ]:
filename = 'betas.csv'
with open(filename, 'w') as f:
   writer = csv.writer(f, delimiter=',')
   writer.writerows(betas)

In [ ]:
filename = 'te_params.csv'
with open(filename, 'w') as f:
   writer = csv.writer(f, delimiter=',')
   writer.writerows(te_params)

In [ ]:
filename = 'iso_params.csv'
with open(filename, 'w') as f:
   writer = csv.writer(f, delimiter=',')
   writer.writerows(iso_params)

In [ ]:
filename = 'S0s.csv'
with open(filename, 'w') as f:
   writer = csv.writer(f, delimiter=',')
   writer.writerows(S0s)

In [ ]:
LSEs = np.zeros(n)
for j in range(0,n):
    LSEs[j] = utils.LSE(predicts[j], (data_unseen*n_variations)[j])

In [ ]:
LSE_average = np.zeros(n_variations)
for m in range(n_variations):
    LSE_average[m] = np.mean(LSEs[range(m*n_voxels, (m+1)*n_voxels)])

In [ ]:
print("Median LSE = %s"%np.mean(LSEs))

In [ ]:
LSEs